## Demo 
    how model.py aka backend works

    remember to run in the right env / check if the packages is installed

In [1]:
from EMO_AI.model_api import *
from EMO_AI.data_process import *

In [3]:
"""get randomly initialized model, not the pretrained one"""
model = get_model(pretrained=False)

c:\Users\Kelvin\anaconda3\envs\AI_sheng\lib\site-packages\transformers\models\auto\modeling_auto.py:973: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


#### We would have to find the tokenizer to parse mandarin
#### so that we can reuse our trained model
#### or, just re-train one (tho we would have to deal with labeling in that scenario)

In [5]:
a = get_output("幹林老師", model)
a

'surprise'

In [6]:
label2int

{'sadness': 0, 'joy': 1, 'love': 2, 'anger': 3, 'fear': 4, 'surprise': 5}

### Low level ones

#### get the pretrained model

In [ ]:
# change this
PATH = "C:"+chr(92)+"Users"+chr(92)+"Kelvin"+chr(92)+"Desktop"+chr(92)+"ai_sheng"+chr(92)+"emo_0.pt"
model = get_model(PATH, inference_only=False)

# model's architecture should be print
# because of model.eval() (used in the function)

c:\Users\Kelvin\anaconda3\envs\AI_sheng\lib\site-packages\transformers\models\auto\modeling_auto.py:973: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
c:\Users\Kelvin\anaconda3\envs\AI_sheng\lib\site-packages\transformers\optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


#### see if run inference (forward pass) works

In [ ]:
# from 01 notebook
t = "Elvis is the king of rock"
# idk what happened
setup_tokenizer()
tokenizer = get_tokenizer()
rep = model(convert_text_to_tensor(t))

In [ ]:
# it works !
rep

tensor([[-0.1518,  1.4839, -1.5390,  0.5634,  1.1406, -0.9670]],
       grad_fn=<AddmmBackward0>)

#### let's transform the output from tensor to string
#### aka, print the emotion (string) as well

In [ ]:
import torch

In [ ]:
# first, change the output to the label
out = torch.argmax(rep, dim=1)
print(out)

# get from the label2int map
from EMO_AI.model_api import label2int

for key in label2int:
    if label2int[key] == out:
        print("Emotion: %s" % key)
        break

tensor([1])
Emotion: joy


In [ ]:
def print_emotion(output):
    # output = model.forward(input)
    idx = torch.argmax(output, dim=1)
    from EMO_AI.model_api import label2int
    for key in label2int:
        if label2int[key] == idx:
            print("Emotion: %s" % key)
            break
    return key

print_emotion(rep)

Emotion: joy


'joy'

## High level ones

In [1]:
text = "I try so hard to feed her appetite"

In [2]:
from EMO_AI.all import *
model = get_model(pretrained=False)
get_output(text, model, return_tensor=True, print_emo=True)

c:\Users\Kelvin\anaconda3\envs\AI_sheng\lib\site-packages\transformers\models\auto\modeling_auto.py:973: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Emotion: fear


tensor([[-0.0466, -0.2029, -0.0012, -0.0182,  0.2248, -0.0831]])

## Try to reuse the cache
#### we ended up just using @lru_cache defined in functools

In [1]:
from EMO_AI.all import *
PATH = "C:"+chr(92)+"Users"+chr(92)+"Kelvin"+chr(92)+"Desktop"+chr(92)+"ai_sheng"+chr(92)+"emo_0.pt"
%time model = get_model(PATH, inference_only=False)

c:\Users\Kelvin\anaconda3\envs\AI_sheng\lib\site-packages\transformers\models\auto\modeling_auto.py:973: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Wall time: 7.28 s


c:\Users\Kelvin\anaconda3\envs\AI_sheng\lib\site-packages\transformers\optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [2]:
%time get_output("this love has taken its toll on me", model)

Wall time: 1.79 s


'sadness'

In [3]:
%time get_output("this love has taken its toll on me", model)

Wall time: 1.6 s


'sadness'

In [4]:
%time t = get_tokenizer()

Wall time: 0 ns


In [5]:
%time get_output("I have not choice, cause i won't say goodbye anymore", model)

Wall time: 1.37 s


'sadness'

In [6]:
%time get_output("What you mean by it's all sadness?", model)

Wall time: 1.43 s


'sadness'

In [7]:
%time get_output("!@#$^&*(*&^%$RFGHUY", model)

Wall time: 1.4 s


'anger'

In [8]:
%time get_output("happy", model)

Wall time: 1.43 s


'joy'

### Conclusion, just declare the tokenizer outside
### and pass it would be much faster than messing 
### around the way to reuse cache

In [9]:
# fastest than above ones without passing "tokenizer" argument 
%time get_output("!@#$^&*(*&^%$RFGHUY", model, t)

Wall time: 1.35 s


'anger'